In [67]:
# INITIAL IMPORTS
import numpy as np
import pandas as pd

# READ IN FILE
df = pd.read_csv('who_suicide_statistics_modified3.csv')
# CHECK READ CORRECTLY
df.head()

,country,year,sex,age,suicides_no,population,HDI for year,gdp_for_year ($)
0,Albania,1987,male,15-24 years,21,312900,NaN,"2,156,624,900"
1,Albania,1987,male,35-54 years,16,308000,NaN,"2,156,624,900"
2,Albania,1987,female,15-24 years,14,289700,NaN,"2,156,624,900"
3,Albania,1987,male,75+ years,1,21800,NaN,"2,156,624,900"
4,Albania,1987,male,25-34 years,9,274300,NaN,"2,156,624,900"


In [68]:
# Q1 - LIST ALL COUNTRIES
df['country'].unique()

array(['Albania', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba',
       'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain',
       'Barbados', 'Belarus', 'Belgium', 'Belize',
       'Bosnia and Herzegovina', 'Brazil', 'Bulgaria', 'Cabo Verde',
       'Canada', 'Chile', 'Colombia', 'Costa Rica', 'Croatia', 'Cuba',
       'Cyprus', 'Czech Republic', 'Denmark', 'Dominica', 'Ecuador',
       'El Salvador', 'Estonia', 'Fiji', 'Finland', 'France', 'Georgia',
       'Germany', 'Greece', 'Grenada', 'Guatemala', 'Guyana', 'Hungary',
       'Iceland', 'Ireland', 'Israel', 'Italy', 'Jamaica', 'Japan',
       'Kazakhstan', 'Kiribati', 'Kuwait', 'Kyrgyzstan', 'Latvia',
       'Lithuania', 'Luxembourg', 'Macau', 'Maldives', 'Malta',
       'Mauritius', 'Mexico', 'Mongolia', 'Montenegro', 'Netherlands',
       'New Zealand', 'Nicaragua', 'Norway', 'Oman', 'Panama', 'Paraguay',
       'Philippines', 'Poland', 'Portugal', 'Puerto Rico', 'Qatar',
       'Republic of Korea', 'Romania', '

In [69]:
# Q2 - SHOW PROBLEMS WITH DATA
# COLUMNS WITH MISSINGS
df.loc[:, df.isnull().any()].columns

Index(['suicides_no', 'HDI for year'], dtype='object')

In [70]:
# RELATIVE FREQ OF MISSINGS
df.isna().sum()/(len(df))*100

country                0.000000
year                   0.000000
sex                    0.000000
age                    0.000000
suicides_no           15.319684
population             0.000000
HDI for year          69.942529
 gdp_for_year ($)      0.000000
dtype: float64

In [71]:
# RENAME GDP FOR EASE OF ACCESS
df.rename(columns={' gdp_for_year ($) ':'gdp'}, inplace=True)

In [72]:
# CHECK DATA FORMATS
df.dtypes

country          object
year              int64
sex              object
age              object
suicides_no      object
population        int64
HDI for year    float64
gdp              object
dtype: object

In [73]:
# Q3 - DATA CLEANING
# REMOVE COLUMN HDI for year
del df['HDI for year']

# SUICIDE_NO MISSING DATA
# REMOVE ROWS WITH MISSING VALUES
df = df.dropna(subset=['suicides_no'])

# REMOVE ROWS WITH NULL
df.loc[df['suicides_no'] == 'Null']
df = df[df['suicides_no'] != 'Null']

# REMOVE ROWS WITH UNKNOWN 
df.loc[df['suicides_no'] == 'Unknown']
df = df[df['suicides_no'] != 'Unknown']

In [74]:
# CHANGE DATA TYPES
# SUICIDE_NO TO INT
df['suicides_no'] = df['suicides_no'].astype(int)

#GDP TO INT
df['gdp'] = df['gdp'].str.replace(',', '').astype(float)

In [75]:
# CHECK DATA TYPES
df.dtypes

country         object
year             int64
sex             object
age             object
suicides_no      int32
population       int64
gdp            float64
dtype: object

In [76]:
# CHECK FOR MISSINGS
df.isna().any().any()

False

In [77]:
# Q4 - ADD SUICIDES/100K COLUMN
df['suicides/100k'] = df['suicides_no'] / (df['population'] / 100000)
df.head()

,country,year,sex,age,suicides_no,population,gdp,suicides/100k
0,Albania,1987,male,15-24 years,21,312900,2.156625e+09,6.711409
1,Albania,1987,male,35-54 years,16,308000,2.156625e+09,5.194805
2,Albania,1987,female,15-24 years,14,289700,2.156625e+09,4.832585
3,Albania,1987,male,75+ years,1,21800,2.156625e+09,4.587156
4,Albania,1987,male,25-34 years,9,274300,2.156625e+09,3.281079


In [78]:
# Q5 - ADD GENERATION COLUMN 
# CREATE NEW COLUMN FOR AGE LOWER BOUND
df['age_lower_bound'] = np.where(df['age'].str[0:2] == '5-', df['age'].str[0], df['age'].str[0:2])
df['age_lower_bound'] = df['age_lower_bound'].astype(int)

In [79]:
# CREATE NEW COLUMN FOR GENERATION
bins = [1883, 1901, 1928, 1946, 1965, 1981, 1996, 2011, 2025]
names = ['Lost Generation', 'G.I. Generation', 'Silent', 'Boomers', 'Generation X', 'Milennials', 'Generation Z', 'Generation A']

df['generation'] = pd.cut((df['year'] - df['age_lower_bound']), bins, labels=names)

In [80]:
df.head()

,country,year,sex,age,suicides_no,population,gdp,suicides/100k,age_lower_bound,generation
0,Albania,1987,male,15-24 years,21,312900,2.156625e+09,6.711409,15,Generation X
1,Albania,1987,male,35-54 years,16,308000,2.156625e+09,5.194805,35,Boomers
2,Albania,1987,female,15-24 years,14,289700,2.156625e+09,4.832585,15,Generation X
3,Albania,1987,male,75+ years,1,21800,2.156625e+09,4.587156,75,G.I. Generation
4,Albania,1987,male,25-34 years,9,274300,2.156625e+09,3.281079,25,Boomers


In [81]:
# CHECK
df['generation'].unique()

[Generation X, Boomers, G.I. Generation, Silent, Milennials, Generation Z]
Categories (6, object): [G.I. Generation < Silent < Boomers < Generation X < Milennials < Generation Z]

In [82]:
# DELETE AGE_LOWER_BOUND COLUMN
del df['age_lower_bound']

In [83]:
df.head()

,country,year,sex,age,suicides_no,population,gdp,suicides/100k,generation
0,Albania,1987,male,15-24 years,21,312900,2.156625e+09,6.711409,Generation X
1,Albania,1987,male,35-54 years,16,308000,2.156625e+09,5.194805,Boomers
2,Albania,1987,female,15-24 years,14,289700,2.156625e+09,4.832585,Generation X
3,Albania,1987,male,75+ years,1,21800,2.156625e+09,4.587156,G.I. Generation
4,Albania,1987,male,25-34 years,9,274300,2.156625e+09,3.281079,Boomers


In [86]:
# Q6 - ADD GDP_PER_CAPITA COLUMN
df['gdp_per_capita'] = df['gdp'] / df['population']
df.head()

,country,year,sex,age,suicides_no,population,gdp,suicides/100k,generation,gdp_per_capita
0,Albania,1987,male,15-24 years,21,312900,2.156625e+09,6.711409,Generation X,6892.377437
1,Albania,1987,male,35-54 years,16,308000,2.156625e+09,5.194805,Boomers,7002.028896
2,Albania,1987,female,15-24 years,14,289700,2.156625e+09,4.832585,Generation X,7444.338626
3,Albania,1987,male,75+ years,1,21800,2.156625e+09,4.587156,G.I. Generation,98927.747706
4,Albania,1987,male,25-34 years,9,274300,2.156625e+09,3.281079,Boomers,7862.285454


In [87]:
df.dtypes

country             object
year                 int64
sex                 object
age                 object
suicides_no          int32
population           int64
gdp                float64
suicides/100k      float64
generation        category
gdp_per_capita     float64
dtype: object

In [88]:
# Q7 - RANK COUNTRIES BY TOTAL SUICIDES
df_country_suicides = df.groupby(['country'])['suicides_no'].sum().to_frame(name = 'country_suicides').sort_values(['country_suicides'])

In [89]:
df_country_suicides.head()

,country_suicides
country,
San Marino,4
Antigua and Barbuda,11
Maldives,20
Macau,27
Oman,33


In [90]:
# Q8 - FIND TOTAL SUICIDES BY CONTINENT
# READ IN CONTINENTS DATASET
df_continents = pd.read_csv('continent_mapping.csv')
df_continents.head()

,country,continent
0,Albania,Europe
1,Antigua and Barbuda,North America
2,Argentina,South America
3,Armenia,Asia
4,Aruba,North America


In [91]:
# MERGE WITH EXISTING DATASET ON COUNTRY
df = pd.merge(df, df_continents, on="country")
df.head()

,country,year,sex,age,suicides_no,population,gdp,suicides/100k,generation,gdp_per_capita,continent
0,Albania,1987,male,15-24 years,21,312900,2.156625e+09,6.711409,Generation X,6892.377437,Europe
1,Albania,1987,male,35-54 years,16,308000,2.156625e+09,5.194805,Boomers,7002.028896,Europe
2,Albania,1987,female,15-24 years,14,289700,2.156625e+09,4.832585,Generation X,7444.338626,Europe
3,Albania,1987,male,75+ years,1,21800,2.156625e+09,4.587156,G.I. Generation,98927.747706,Europe
4,Albania,1987,male,25-34 years,9,274300,2.156625e+09,3.281079,Boomers,7862.285454,Europe


In [92]:
# CREATE NEW DATAFRAME WITH CONTINENT AND SUICIDES
df_continent_suicides = df.groupby(['continent'])['suicides_no'].sum().to_frame(name = 'continent_suicides').sort_values(['continent_suicides'])
df_continent_suicides.head()

,continent_suicides
continent,
Africa,10089
Oceania,72379
South America,353165
North America,1004670
Asia,1150065


In [93]:
# Q9 - CORRELATIONS
df.corr()

,year,suicides_no,population,gdp,suicides/100k,gdp_per_capita
year,1.000000,-0.002862,0.008520,0.101099,-0.043834,0.190080
suicides_no,-0.002862,1.000000,0.538977,0.390525,0.284362,-0.003040
population,0.008520,0.538977,1.000000,0.705792,-0.077188,-0.038280
gdp,0.101099,0.390525,0.705792,1.000000,-0.025837,0.176341
suicides/100k,-0.043834,0.284362,-0.077188,-0.025837,1.000000,0.129824
gdp_per_capita,0.190080,-0.003040,-0.038280,0.176341,0.129824,1.000000


In [94]:
df.to_csv('df.csv', index=False)